In [1]:
# '''
# Goto config ->config.yaml
# Paste below code

#model_trainer:
#   root_dir: artifacts/model_trainer
#   train_data_path: artifacts/data_transformation/train.csv
#   test_data_path: artifacts/data_transformation/test.csv
#   model_name: model.joblib

# '''

In [2]:
# '''
# Goto params.yaml
# Paste below code

# ElasticNet:
#   alpha: 0.2
#   l1_ratio:0.1 

In [3]:
import os

In [4]:
%pwd

'd:\\Project\\Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Project\\Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment'

# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

# Config folder 
1. Copy get_model_trainer_config function 
2. Goto config folder src->wine_quality->config->configuration.py paste under ConfigurationManager class
3. Import ModelTrainerConfig from textSummarizer.entity using this line  
from wine_quality.entity import ModelTrainerConfig

In [8]:
from wine_quality.constants import *
from wine_quality.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        
        )

        return model_trainer_config

# Component folder
1. Copy entire ModelTrainer class
2. Goto Component folder [ src-> textSummarizer -> components - > model_trainer.py ]
2. Paste ModelTrainer class in model_trainer.py
3. Import ModelTrainerConfig from entity using this line 

from textSummarizer.entity import ModelTrainerConfig

In [10]:
import pandas as pd
import os
from wine_quality import logger
from sklearn.linear_model import ElasticNet
import joblib

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

# Pipeline folder

1. Goto Component folder [ src-> textSummarizer -> pipeline - > stage_04_model_trainer.py ]

2. Paste below code in stage_04_model_trainer.py

In [12]:
# ''' 
# Example
# from wine_quality import logger
# from wine_quality.config import ConfigurationManager
# from wine_quality.components.model_trainer import ModelTrainer
# class ModelTrainerTrainingPipeline:
#     def __init__(self):
#         pass
#     def main(self):
        
# '''

3. Copy try block code 
4. Paste in main function in stage_04_model_trainer.py

5. Import ModelTrainerConfig from config.configuration using this line  
from wine_quality.config.configuration import ConfigurationManager

6. Import ModelTrainer  from  components.model_trainer  using this line  
from wine_quality.components.model_trainer  import ModelTrainer 

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-17 21:56:23,649] - wine_quality - INFO - common file - LineNum:27   - yaml file : config\config.yaml loaded successfully
[2023-09-17 21:56:23,668] - wine_quality - INFO - common file - LineNum:27   - yaml file : params.yaml loaded successfully
[2023-09-17 21:56:23,678] - wine_quality - INFO - common file - LineNum:27   - yaml file : schema.yaml loaded successfully
[2023-09-17 21:56:23,682] - wine_quality - INFO - common file - LineNum:43   - Created Directory At: artifacts
[2023-09-17 21:56:23,687] - wine_quality - INFO - common file - LineNum:43   - Created Directory At: artifacts/model_trainer


In [14]:
# '''
# Example
# from wine_quality.config import ConfigurationManager
# from wine_quality.conponents.model_trainer import ModelTrainer
# from wine_quality import logger


# class ModelTrainerTrainingPipeline:
#     def __init__(self):
#         pass

#     def main(self):
#         config = ConfigurationManager()
#         model_trainer_config = config.get_model_trainer_config()
#         model_trainer_config = ModelTrainer(config=model_trainer_config)
#         model_trainer_config.train()
# '''

In [15]:
# '''
# 1. Goto main.py and paste below code
# STAGE_NAME = "Model Trainer stage"
# try: 
#    logger.info(f"*******************")
#    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
#    model_trainer = ModelTrainerTrainingPipeline()
#    model_trainer.main()
#    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
# except Exception as e:
#         logger.exception(e)
#         raise e

# 2. Run main.py 
# '''